# Ta feng Data

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
root_dir = "./data/ta_feng/"
csv_file = "ta_feng_all_months_merged.csv"

df = pd.read_csv(root_dir+file)

In [4]:
df

,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
0,11/1/2000,1104905,45-49,115,110411,4710199010372,2,24,30
1,11/1/2000,418683,45-49,115,120107,4710857472535,1,48,46
2,11/1/2000,1057331,35-39,115,100407,4710043654103,2,142,166
3,11/1/2000,1849332,45-49,Others,120108,4710126092129,1,32,38
4,11/1/2000,1981995,50-54,115,100205,4710176021445,1,14,18
...,...,...,...,...,...,...,...,...,...
817736,2/28/2001,312790,35-39,114,530501,4713317035042,2,80,118
817737,2/28/2001,57486,40-44,115,530209,4710731060124,1,40,55
817738,2/28/2001,733526,>65,Unknown,510539,4716340052307,1,78,115
817739,2/28/2001,173704,45-49,115,520457,4714276145315,1,90,96


In [5]:
customer_id = set(df["CUSTOMER_ID"])

In [6]:
trans = []
cnt = 0

for i in customer_id:
    customer = df[df["CUSTOMER_ID"] == i]
    trans_date = list(set(customer["TRANSACTION_DT"]))
    # print(i, "\n", trans_date)
    
    if len(trans_date) < 4:
        cnt += 1
        continue

    for d in trans_date:
        product = customer[customer["TRANSACTION_DT"] == d]
        if len(set(product["PRODUCT_ID"])) > 2:
            trans.append(set(product["PRODUCT_ID"]))
        else:
            cnt += 1

In [7]:
import pickle

filename = "transaction.dat"

In [8]:
filehandler = open(root_dir+filename, "wb") 
pickle.dump(trans, filehandler)

In [9]:
f = open(filename,"rb")
trans_from_pickle = pickle.load(f)

## # transactions

In [10]:
len(trans_from_pickle)

59191

## # items

In [11]:
prod_set = set()
for i in trans_from_pickle:
    prod_set |= i
    
print(len(prod_set))

21886


## # removed transactions

In [12]:
cnt

46373

## Average # item in a transaction

In [13]:
mean = 0
for i in trans_from_pickle:
    mean += len(i)
    
mean /= len(trans_from_pickle)
mean

8.334865097734452

## Make triple dataroot_dirBASKET-SENSITIVE FACTORIZATION MACHINE(BFM)